In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.io as pio
import pandas as pd
from pathlib import Path
import datetime
pio.renderers.default = 'iframe'

data_folder = Path('data/')

In [4]:
# remove test station, only look at first half of 2023 to keep the data dimensionality manageable
bike_df = pd.read_parquet(data_folder/'dublinbikes/combined_data_weather_population.parquet', filters=[
    ('time', '>=', datetime.datetime(2023,1,1)),
    ('time', '<=', datetime.datetime(2023,6,1)),
    ('station_id', '!=', 507)
])
bike_df['half_hour'] = bike_df['time'].dt.minute//30*30 == 30
bike_df['empty_station'] = bike_df['available_bikes'] == 0
bike_df['full_station'] = bike_df['available_bike_stands'] == 0
bike_df['needs_attention'] = bike_df['full_station'] | bike_df['empty_station']
bike_df = bike_df.loc[(~pd.isna(bike_df['rain'])) & (~bike_df['half_hour'])] 

print(bike_df.shape)
bike_df.head()

(411250, 29)


,station_id,time,last_updated,name,bike_stands,available_bike_stands,available_bikes,status,address,latitude,...,day_of_month,hour_of_day,rain,temp,rhum,population,half_hour,empty_station,full_station,needs_attention
0,1,2023-01-01 00:00:03,2022-12-31 23:59:39,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,0,0.3,6.8,91.0,5522.0,False,True,False,True
2,1,2023-01-01 01:00:03,2023-01-01 00:50:23,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,1,0.0,6.2,86.0,5522.0,False,True,False,True
4,1,2023-01-01 02:00:03,2023-01-01 01:50:55,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,2,0.0,4.5,91.0,5522.0,False,True,False,True
6,1,2023-01-01 03:00:03,2023-01-01 02:51:28,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,3,0.1,4.0,93.0,5522.0,False,True,False,True
8,1,2023-01-01 04:00:02,2023-01-01 03:52:00,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,...,1,4,0.0,2.5,96.0,5522.0,False,True,False,True


Dashboard - 
Intended audience is a decision maker with regards to adjustments of bike levels within the dublin bike network. The user should be able to see demand at various stations. Crucially, they should be aware of when stations are full or empty so that they can make decisions around bike redistribution / replenishment

Controls:

- date & time selector
- station selector

Charts:

- Weekly rentals - line chart of the bike rentals in the selected week in selected station
- Weekly weather report - Line chart of temperature over the week, bar chart of rainfall (same for all stations)
- Station map - Plot points for each station - represent the number of bikes available with piont size and colour - clickable to select station
    - Colour choice is blue for staions that do not need attention and red for those that do, red naturally draws the eye to the important information
    - Size is artificially inflated for stations with 0 bikes available, again to draw the eye 

The layout is specified with bootstrap components so that figures can be arranged into rows and columns with specified widths, and the dash_bootstrap_components css themes can be easily applied. The Lux theme was chosen for its simplicity and minimalism. Tufte typically argues for minimising ink-usage, so a dashboard theme without extraneous decoration will not get in the way of conveying the information contained in the graphics on the dashboard.

In [8]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.themes.LUX])

@app.callback(
    Output('map', 'figure'),
    [Input('time-selector', 'value')] ,
    [State('map', 'relayoutData')]
)
def update_map(selected_time, map_settings):
    selected_time = datetime.datetime.fromtimestamp(selected_time) 
    selected_data = bike_df[bike_df['time'].between(selected_time - datetime.timedelta(minutes=10), selected_time+datetime.timedelta(minutes=10))]
    
    # show stations with no bikes - make 0s large as they are important information
    selected_data['point_size'] = selected_data['available_bikes'].apply(lambda x: x + 10 if x == 0 else x)
    
    fig = px.scatter_mapbox(
        selected_data, 
        lat='latitude', 
        lon='longitude', 
        size='point_size', 
        color='needs_attention', 
        color_discrete_map={False: '#005AB5', True: '#DC3220'}, 
        hover_data=['station_id', 'address', 'available_bikes', 'bike_stands'], 
        mapbox_style='carto-positron'
    )
    
    fig.update(layout_showlegend=False)
    
    lat_range = [selected_data['latitude'].min(), selected_data['latitude'].max()]
    lon_range = [selected_data['longitude'].min(), selected_data['longitude'].max()]

    # if user moves the map keep their settings on update
    if map_settings and 'mapbox.center' in map_settings:
        center_lat = map_settings['mapbox.center']['lat']
        center_lon = map_settings['mapbox.center']['lon']
    else:
        center_lat = (lat_range[0] + lat_range[1]) / 2
        center_lon = (lon_range[0] + lon_range[1]) / 2 
        
    if map_settings and 'mapbox.zoom' in map_settings:
        zoom_level = map_settings['mapbox.zoom']
    else:
        zoom_level = 12
        
    fig.update_layout(mapbox=dict(center=dict(lat=center_lat, lon=center_lon), zoom=zoom_level),  margin=dict(l=0, r=0, t=0, b=0))  
    
    return fig
    
@app.callback(
    Output('ts', 'figure'),
    [Input('station-selector', 'value'),
     Input('time-selector', 'value')],
)
def update_ts(station, selected_time):
    # update the number of rentals per week chart
    selected_time = datetime.datetime.fromtimestamp(selected_time)
    start_of_week = selected_time - datetime.timedelta(days=selected_time.weekday())
    start_of_week = start_of_week.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_week = start_of_week + datetime.timedelta(days=7)
    
    selected_data = bike_df[(bike_df['station_id'] == station) & (bike_df['time'] <= end_of_week) & (bike_df['time'] >= start_of_week)]
    
    fig = px.line(selected_data, x='time', y='available_bikes', markers=True, title=f'Weekly Bike availability')
    fig.update_traces(line=dict(color='black', width=0.5))

    # show where exactly the time and date selector is within the week
    fig.add_shape(
        dict(type="line", x0=selected_time, x1=selected_time, y0=0, y1=1, xref='x', yref='paper', line=dict(color='red', width=1))
    )
    
    days = [start_of_week + datetime.timedelta(days=x) for x in range(8)]
    for day in days:
        fig.add_annotation(
            dict(x=day, y=1.02, text=day.strftime("%A"), showarrow=False, xref='x', yref='paper', font=dict(color='black', size=10))
        )
    return fig


@app.callback(
    Output('weather', 'figure'),
    [Input('station-selector', 'value'),
     Input('time-selector', 'value')],
)
def update_weather(station, selected_time):
    # update the weekly weather chart
    selected_time = datetime.datetime.fromtimestamp(selected_time)
    start_of_week = selected_time - datetime.timedelta(days=selected_time.weekday())
    start_of_week = start_of_week.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_week = start_of_week + datetime.timedelta(days=7)
    
    selected_data = bike_df[(bike_df['station_id'] == station) & (bike_df['time'] <= end_of_week) & (bike_df['time'] >= start_of_week)]
    
    fig = px.line(selected_data, x='time', y='temp', labels={'temp': 'Temperature (°C)'}, line_shape='linear')
    
    fig.add_bar(x=selected_data['time'], y=selected_data['rain'], name='Rainfall', yaxis='y2', marker=dict(color='rgba(0,0,0,0)', line=dict(color='black', width=0.5)))
    
    fig.update(layout_showlegend=False)
    
    fig.add_shape(
        dict(type="line", x0=selected_time, x1=selected_time, y0=0, y1=1, xref='x', yref='paper', line=dict(color='red', width=1))
    )
    
    days = [start_of_week + datetime.timedelta(days=x) for x in range(8)]
    for day in days:
        fig.add_annotation(
            dict(x=day, y=1.02, text=day.strftime("%A"), showarrow=False, xref='x', yref='paper', font=dict(color='black', size=10))
        )
        
    fig.update_layout(
        yaxis=dict(title='Temperature (°C)'),
        yaxis2=dict(title='Rainfall (mm)', overlaying='y', side='right'),
        title='Weekly Weather',
        xaxis=dict(title='Date')
    )
    return fig
    

@app.callback(
    Output('station-selector', 'value'),
    [Input('map', 'clickData')]
)
def update_station_selector(click_data):
    if click_data is not None and 'points' in click_data:
        selected_station = click_data['points'][0]['customdata'][0]
        return selected_station
    else:
        return bike_df['station_id'].unique()[0]


@app.callback(
    Output('slider-output', 'children'),
    Input('time-selector', 'value')
)
def update_slider_label(value):
    return datetime.datetime.utcfromtimestamp(value).replace(second=0).strftime("%Y-%m-%d %H:%M:%S")
    

In [9]:
min_timestamp = bike_df['time'].min().timestamp()
max_timestamp = bike_df['time'].max().timestamp()
app.layout = dbc.Container(children=[
    dbc.Row(
        html.H1(children="Dublin Bikes"),
    ),
    
    dbc.Row([
        dbc.Col([
            html.Label('Select Station:'),
            dcc.Dropdown(
                id='station-selector',
                options=[{'label': address, 'value': id} for address, id in zip(bike_df['address'].unique(), bike_df['station_id'].unique())],
                value=bike_df['station_id'].unique()[0]
            )
        ], width=3),
        
        dbc.Col([
            html.Label('Select Date and Time:'),
            dcc.Slider(
                id='time-selector',
                step=3600, 
                min=min_timestamp,
                max=max_timestamp,
                marks=None,
                value=min_timestamp
            ),
            html.Div(id='slider-output'),
        ], width=9)
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id='ts'),
            dcc.Graph(id='weather'),
        ], width=7),
        
        dbc.Col([dcc.Graph(id='map')], width=5),
    ])
])

In [10]:
app.run_server(debug=True, use_reloader=False, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/


/tmp/ipykernel_627851/3149875911.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

